### Big data course project
<strong>T7: Forecasting demand -- augmentation of the data</strong>

Jovana Videnovic & Haris Kupinic

In [194]:
import pandas as pd
from pathlib import Path

In [195]:
data_path = Path("/d/hpc/home/jv8043/BD/project/T7/data")
add_data_path = Path("/d/hpc/home/jv8043/BD/project/T5/add_data")

In [196]:
df_name = "dc_fhvhv"

In [197]:
df = pd.read_csv(data_path / f"{df_name}.csv")

In [198]:
# add covid column based on pickup_datetime
df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"])

In [199]:
df["covid"] = (
    ((df["pickup_datetime"].dt.year == 2020) & (df["pickup_datetime"].dt.month > 3)) |
    ((df["pickup_datetime"].dt.year == 2020) & (df["pickup_datetime"].dt.month == 3) & (df["pickup_datetime"].dt.day >= 15)) |
    ((df["pickup_datetime"].dt.year == 2021) & (df["pickup_datetime"].dt.month < 4))
)


In [200]:
holidays_df = pd.read_csv(add_data_path / "holidays.csv")

In [201]:
holiday_date_list = pd.to_datetime(holidays_df["Date"]).dt.date.tolist()
print(f"Number of holidays: {len(holiday_date_list)}")

Number of holidays: 75


In [202]:
df["holiday"] = df["pickup_datetime"].dt.date.isin(
    holiday_date_list
)

In [203]:
weather_df = pd.read_csv(add_data_path / "weather.csv")
weather_df["DATE"] = pd.to_datetime(weather_df["DATE"])
# rename columns to match df
weather_df.rename(columns={
    "DATE": "pickup_datetime"}, inplace=True)

In [204]:
# drop columns 
weather_df = weather_df.drop(columns=["STATION","NAME"])

In [205]:
for c in ["AWND","PRCP","SNOW","SNWD","TAVG","TMAX","TMIN"]:
    weather_df[c] = weather_df[c].astype(float)

In [206]:
# join weather data with df based on pickup_datetime
df = df.merge(weather_df, on="pickup_datetime", how="left")

In [207]:
del df["TAVG"]
df = df.fillna({
    "AWND": 0.0,
})

In [208]:
df["is_school_season"] = df["pickup_datetime"].between("2019-09-01", "2020-03-15") | \
                         df["pickup_datetime"].between("2021-09-01", "2022-06-30") | \
                         df["pickup_datetime"].between("2022-09-01", "2023-06-30") | \
                         df["pickup_datetime"].between("2023-09-01", "2024-06-30") | \
                         df["pickup_datetime"].between("2024-09-01", "2025-06-30")


In [209]:
df["is_spring_break"] = (
    df["pickup_datetime"].between("2020-03-09", "2020-03-15") |
    df["pickup_datetime"].between("2021-03-15", "2021-03-21") |
    df["pickup_datetime"].between("2022-04-11", "2022-04-17") |
    df["pickup_datetime"].between("2023-04-03", "2023-04-09") |
    df["pickup_datetime"].between("2024-03-25", "2024-03-31") |
    df["pickup_datetime"].between("2025-03-24", "2025-03-30")
)


In [210]:
df.to_csv(data_path / f"{df_name}_augmented.csv", index=False)

In [211]:
display(df.head())

,pickup_datetime,ride_count,covid,holiday,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,is_school_season,is_spring_break
0,2019-03-01,837591,False,False,0.0,0.11,1.4,1.2,35.0,26.0,False,False
1,2019-03-02,900381,False,False,0.0,0.53,4.0,3.9,41.0,31.0,False,False
2,2019-03-03,796165,False,False,0.0,0.52,3.0,1.2,42.0,33.0,False,False
3,2019-03-04,621067,False,False,0.0,0.29,2.0,3.9,38.0,28.0,False,False
4,2019-03-05,710006,False,False,0.0,0.00,0.0,2.0,32.0,20.0,False,False
